In [62]:
import os
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
model_file = "zh_word2vec.model"
model = Word2Vec.load(model_file)

model.wv.most_similar("说", topn=30)


[('为什么', 0.8298794627189636),
 ('问', 0.8270461559295654),
 ('说道', 0.8173126578330994),
 ('为何', 0.8128331899642944),
 ('告诉', 0.7882134318351746),
 ('吗', 0.7878919839859009),
 ('的话', 0.7855218648910522),
 ('所说', 0.7767419219017029),
 ('觉得', 0.7696527242660522),
 ('讲', 0.7678855657577515),
 ('怎么样', 0.7658766508102417),
 ('声称', 0.764196515083313),
 ('以为', 0.7633250951766968),
 ('问道', 0.7630106210708618),
 ('认为', 0.758924126625061),
 ('中说', 0.751205325126648),
 ('时说', 0.7498580813407898),
 ('写道', 0.7494862079620361),
 ('告诫', 0.7469834089279175),
 ('说起', 0.7460322380065918),
 ('确信', 0.7414821982383728),
 ('质问', 0.7364925146102905),
 ('谈论', 0.7361835241317749),
 ('感叹', 0.7359782457351685),
 ('说出', 0.7354288101196289),
 ('谁', 0.7353894710540771),
 ('称', 0.7324063181877136),
 ('却说', 0.7310951948165894),
 ('也许', 0.7306363582611084),
 ('或许', 0.730185866355896)]

In [80]:
from collections import defaultdict
def most_similar(start_word, max_layer=7):
    word_paths = [[start_word]]
    occurences = defaultdict(int)
    searched_words = {}
    similar_words_in_layers = [start_word]
    while True:
        if len(word_paths) > 0:
            word_path = word_paths.pop(0)
        else:
            return occurences
        word = word_path[-1]
        if word in searched_words:
            similar_words = searched_words[word]
        else:
            similar_words_with_possibility = model.wv.most_similar(word, topn=20)
            similar_words = [word for word, posibility in similar_words_with_possibility]
        for similar_word in similar_words:
            if start_word == similar_word:
                if word not in similar_words_in_layers:
                    #similar_words_in_layers.append(word)
                    for word in word_path:
                        occurences[word] += 1
            else:
                new_word_path = word_path+[similar_word]
                if similar_word not in word_path and len(new_word_path) <= max_layer:
                    word_paths.append(new_word_path)
                
        searched_words[word] = similar_words


In [ ]:
most_similar_words = most_similar("说")
from collections import Counter
counter = Counter(most_similar_words)

In [ ]:
counter.most_common(15)

In [ ]:
model.wv.most_similar("说", topn=15)